# 3 监督学习
监督学习的目标是通过构建自变量与因变量间关系的拟合模型，解释数据的规律性，以便预期未来结果的。即：当 $y$ 已知时，求解功能函数 $F$ ，使 $y = F(x)$ ；现实中大多无法绝对相等，只能尽可能大的拟合$y$中的绝大多数。

根据 $y$ 的差异划分为：
- **回归**（$y$ 无限集）
- **分类**（$y$ 有限集）两大类。

其中，分类问题常见的是二分类问题（比如垃圾邮件识别），但也可以拥有任意数量的类，属于多类分类问题；前者模型称作二分类器，后者称作多类分类器（多项式分类器）。

当处理多分类问题时，如随机森林、朴素贝叶斯等分类器支持多分类问题；存在类似SVM、线性分类器等严格的二分类器，可使用如下策略：
- **一对所有（OvA）**：创建与类别数$N$相等的分类器数，每个二分类器训练时只设定其中一个类别为正例，其余均为负例，选出决策分数最高的分类器所对应的类。
- **一对一（OvO）**：将类别数量$N$两两配对作为训练子集单独训练二分类器，共计训练$\frac{N*(N-1)}{2}$个分类器，选出全部分类器中类别胜出最多的类。

根据 $F$ 的差异划分为：
- **参数化模型**：模型有预定义的结构，需要评估具体的参数值，如逻辑回归、svm等
- **非参数化模型**：模型预先没有具体的结构，如knn、决策树等

此外，实际应用中，多数监督学习算法都支持处理回归&分类问题。

## 3.1 参数化模型
如前所述，模型最重要的三个要素：模型假设、损失函数、寻解算法，故我们将模型抽象为$F(X)$，此处先聚焦共性较强的损失函数与寻解算法。

### 3.1.1 回归问题的损失函数
损失函数，表达了建模者希望模型达成的目标。针对回归问题，我们必然希望模型的预测值$\hat y$与真值$y$越接近越好（最佳情况一摸一样）。首先，想到一个朴素的公式就是 $y - \hat y$ 越小越好，由于模型是希望绝大多数的数据（假设建模记录数为$n$）都被很好的拟合，故期望真值与预估值差异的累计值最小，即：
$$ MBE(平均偏差误差) = \frac{1}{n}\sum_{i = 1}^n(y_i - \hat y_i) $$
从上述公式中不难发现，MBE会受到误差方向的影响，产生正负相互抵消，评估损失的精确度不足的问题，据此改良如下：
$$ MAE(平均绝对值误差,L1Loss) = \frac{1}{n}\sum_{i = 1}^n|y_i - \hat y_i| $$
平均绝对值误差虽然改善了正负偏差相互抵消的问题，但是该函数曲线在零点不可导，为后续寻解算法寻找全局最优解造成了影响，故可再次优化：
$$ RMSE(均方根误差,L2Loss) = \sqrt {MSE(均方误差)} = \sqrt {\frac{1}{n}\sum_{i = 1}^n(y_i - \hat y_i)^2} $$
RMSE函数具有处处可导的特性，相比于MSE数值量级下降很多，使其成为了最常使用的回归问题损失函数；但值得注意的是该函数内层指数函数的特性，对于异常点（真值与预测值偏差极大的）敏感，此时会牺牲对绝大多数正常点参数优化，着重优化对损失函数贡献更高的离群点，所以数据预处理时要多加注意。此外也可以使用如下改良方法：
$$ HuberLoss(平滑绝对误差) = \sum_{i=1}^n \begin{cases} 0.5*(y_i - \hat y_i)^2 & |y_i - \hat y_i|<\delta \\ \delta * (|y_i - \hat y_i|-0.5 \delta ^2) & else \end{cases}$$
当预测值与真值差别过大（大于 $\delta$ ）时，使用L1范数，其余情况使用L2范数，平衡异常点对损失函数值的影响。该函数最常用的形态是：
$$ smooth_{L1}(\delta = 1) = \sum_{i=1}^n \begin{cases} 0.5*(y_i - \hat y_i)^2 & |y_i - \hat y_i|<1 \\ |y_i - \hat y_i|-0.5 & else \end{cases} $$
另外介绍两个特定场景下比较实用的回归问题损失函数：
$$ LogCoshLoss = \sum_{i=1}^n log(cosh(y_i - \hat y_i)) $$
对于较小的 $x$，$log(cosh(x))$ 近似于 $\frac{1}{2}x^2$，对于较大的 $x$，近似于$|x|-log(2)$，具有Huber loss所有的优点且二阶导数处处可微；但当误差极大时，一阶导数和Hessian会变成定值，导致如XGBoost出现缺少分裂点的情况。
$$ QuantileLoss(分位数损失) = \sum_{y_i<\hat y_i}(1-\lambda)|y_i-\hat y_i| + \sum_{y_i\geq\hat y_i}\lambda|y_i - \hat y_i|, \lambda \in [0,1] $$
现实问题中，并不是所有问题都能被模型很好贴合，但是面对某些不太线性但偏向针对某一区间进行预测的问题时，可通过基于对正反误差的重视程度，调节$\lambda$（分位值）给予不同的惩罚，让函数对中值产生偏移，优化解决重点区间。

以上只是回归问题我熟识的损失函数，当然还有许多损失函数未提及，也可以根据实际问题自定义损失函数。

#### 3.1.2 分类问题的损失函数
对于分类问题，使用RMSE及其相关变体理论上是可以的，但是部分模型叠加了比较复杂的符号函数，给此类损失函数造成了较多局部最优解，干扰全局最优的寻找；故并不如下面我们要探讨的损失函数主流。
#### 3.1.2.1 负似然对数
负似然对数损失函数就是似然对数函数的负值，故先阐述**概率与似然函数**的关系。
现用二分类问题进行说明，并做出如下假设：
- 事件有且仅有互斥的正、反两种结果（比如垃圾邮件分类），正例记为1，负例记为0。
- 事件正例发生的概率为：$p$，则事件反例概率为：$1-p$。
- 对事件累计实验$N$次（或收集到$N$条记录），其中正例$m$次，反例$n$次。

则实验的似然函数为：$L(p)=p^m*(1-p)^n$；若为数据记录，似然函数为：$L(p)=C^m_N*p^m*(1-p)^n$。由于系数不改变函数单调性，似然函数的核心便是：$L(p)=p^m*(1-p)^n$。依据统计学中频率学派的观点，由于实验已发生且数据足够充分，故发生这一组合的可能性最高，那么使$L(p)$取最大值的$p$便是真实的事件正例发生概率。

将$p$替换为模型函数的预测结果：$h_\theta(x)$，可获得公式：$ p(\theta) = \begin{cases} h_\theta(x) & y=1 \\ 1-h_\theta(x) & y=0 \end{cases} = h_\theta(x)^y + (1-h_\theta(x))^{(1-y)} $，其中$y$为真实正例或者负例标签。

更新似然函数公式：
$$ L(\theta) = \prod_{i=1}^n h_\theta(x)^{y_i} + (1-h_\theta(x))^{(1-y_i)}  $$

这样就将求解函数参数的问题等价于寻找合适的$ \theta $使似然函数取最大值，相应的便是负似然函数求解最小值，定义损失函数为：
$$ cost(\theta) = -L(\theta) = - \prod_{i=1}^n h_\theta(x)^{y_i} + (1-h_\theta(x))^{(1-y_i)} $$

上述损失函数的求导颇为复杂，对$ L(\theta) $取对数并不影响曲线单调性，即最小值的取值点不变，优化为：
$$ cost(\theta) = -lnL(\theta) = - \sum_{i=1}^n y_i * ln(h_\theta(x)) + (1-y_i)*ln(1-h_\theta(x)) $$

矩阵公式记为：
$$ cost(\theta) = -Y^T * ln(h_\theta(X))-(E - Y)^T * (E - ln(h_\theta(X))) $$

#### 3.1.2.2 交叉信息熵
要理论化的定义交叉信息熵(cross entropy)比较麻烦，所以使用个人粗浅的认知阐述，准确的解答可查阅信息论相关文档。

首先，说明下**信息**，可以说是一种价值度量。以新闻事件来类比，超出人们想象的、小概率发生的事件我们才会将其作为头条新闻，因为该新闻的价值高；类似的，一个系统中，小概率事件蕴含的信息值高。怎么度量呢？因为信息起源于计算机对信息的记录，只有0/1两种标记方案，故以二分类决策推测某事件的具体结果为类比，最优策略总是优先选择最可能发生的结果，此后逐层递减。这个准则下，确定某个事件结果发生所消耗的路径长度就是信息，计算公式：$log_2(\frac{1}{p})$，$p$ 即为某事件的概率（此处我认为$log_n(\frac{1}{p})$,$n$为分叉数）。

**信息熵**：分类系统中多个类别信息值的加权和：$ \sum_{i=1}^n p_i * log_2(\frac{1}{p_i}) $。

**交叉熵**：基于模型预测的事件概率分布$q_i$的信息加权和：$ \sum_{i=1}^n p_i * log_2(\frac{1}{q_i}) $。

**相对熵**：$ 相对熵 = 交叉熵 - 信息熵 = \sum_{i=1}^n p_i * log_2(\frac{p_i}{q_i})$

所以损失函数可以使用相对熵，目标是使得相对熵最小，$ cost(\theta) = \sum_{i=1}^n p_i * log_2(\frac{1}{q_i}) - \sum_{i=1}^n p_i * log_2(\frac{1}{p_i}) $，由于信息熵是常量，故损失函数可以精简为：$ cost(\theta) = \sum_{i=1}^n p_i * log_2(\frac{1}{q_i})= -\sum_{i=1}^n p_i * log_2(q_i) $

令$q_i$为模型$h_\theta(x)$预测结果,则损失函数为：
$$ cost(\theta) = -\sum_{i=1}^n p_i * log_2(q_i) = -\sum_{i=1}^n p_i * [y_i * log_2(h_\theta(x)) + (1-y_i) * log_2(1 - h_\theta(x))] $$

#### 3.1.2.3 指数损失(exponential loss)
*暂时还没研究透彻，adaboost的损失函数*

#### 3.1.2.4 Hinger损失
*暂时还没研究*

#### 3.1.2.5 感知损失（Hinger损失的特例）
*暂时还没研究*

#### 3.1.2.6 铰链损失
*暂时还没研究*

### 3.1.3 寻解算法：最小二乘法（OLS）
**最小二乘法（最小平方法）**：通过最小化误差的平方和寻找数据的最佳匹配函数。对于求解最小二乘法结果的方法有**矩阵解法**和**梯度下降法**。

*以前我以为最小二乘法和梯度下降法是两个方法，其实梯度下降法只是最小二乘法求解的方法之一。*

#### 3.1.3.1 矩阵解法
为方便后续讲解，做出如下前提假设：
- **模型假设**：$ F(x)=a_1*x_1+a_2*x_2+……+a_n*x_n+b=[x_1,x_2,……,x_n,1][a_1,a_2,……,a_n,b]^T=XA $
- **损失函数**：$ L(x,a)=\frac{1}{2n}\sum_{i=1}^n[(a_1*x_{i1}+a_2*x_{i2}+……+a_n*x_{in}+b)-y_i]^2=\frac{1}{2n}(XA-Y)^T(XA-Y) $

依据假设，当损失函数的一阶偏导数（$\partial A$）为为零时，损失函数取最小值，即：
$$ \frac{\partial L}{\partial A}=\frac{\partial L}{\partial a_1}+\frac{\partial L}{\partial a_2}+……+\frac{\partial L}{\partial a_n}=\frac{1}{n}\sum_{i=1}^n(a_1x_{i1}+a_2x_{i2}+……+a_nx_{in})*(x_{i1}+x_{i2}+……+x_{in})=\frac{X^T(XA-Y)}{n}=0 $$
求解：
$$ A = (X^TX)^{-1}X^TY $$
以上的推导过程表明矩阵解法有如下限制：
- 1.系数矩阵$A$必须存在逆矩阵（可舍弃边缘特征使矩阵可逆缓解）。
- 2.系数矩阵$A$的求逆过程的耗时受矩阵规模影响（利用降维处理缓解）。
- 3.模型假设若非线性，求解将十分复杂或难以开展（可以转化为线性关系缓解）。

*（）中是一些预处理方案，优化矩阵解法的适配问题。*

#### 3.1.3.2 梯度下降法
梯度下降法是利用梯度逐步迭代模型参数，使损失函数达到最小值的方法。举例如下：
```
想象你和贝爷一起进行丛林探险，露宿山崖边缘，晨间丛林里雾气弥漫，你们选择下至山下的峡谷补充水源。此时，你们**各选了一个位置**开始下山，你用脚一点点感受被踩踏的岩石，选择合适的落脚点，你总是**优先选择垂直向下（即坡度最陡）**的方向下山，重复如此，最终你们下到了山脚。
```
梯度下降法的过程与之相似，使用相同的假设表达如下：
- **模型假设**：$ F(x)=XA $ 并随机初始化$A$中参数。
- **损失函数**：$ L(x,a)=\frac{1}{2n}(XA-Y)^T(XA-Y) $。
- **偏导数**：$ \frac{\partial L}{\partial A}=\frac{1}{n}X^T(XA-Y) $。
- **梯度下降**：$ A=A-\eta \frac{\partial L}{\partial A},\eta \in (0,1) $。
- **终止条件**：梯度值为零或不可改善时，则达到损失函数最小值，此时的模型参数为最优参数。

学习率（$\eta$）与梯度（$\frac{\partial L}{\partial A}$）共同决定每轮参数矩阵迭代的步长，其中学习率是用户设定值，需注意：
- 模型训练前应特征缩，提高$\eta$作用效果，高效下降。
- $\eta$设定过小，耗时严重；设定过大，反复跳跃难收敛。

与矩阵的解法中求解一阶导数为零不同，由于训练数据$X$，随机参数$A$、学习率$\eta$均为已知项，其目标是将现有的随机参数不断改善成最优函数的接近值，所以突破了矩阵解法的限制，更加通用。但是梯度下降法的主要挑战是复杂的损失函数，除了全局最小值外，还存在多个局部最小值，有时可能永远到不了全局最小值。
在实际使用梯度下降法时，由于计算梯度方式的差异，衍生出三种方法：
- **批梯度下降（BGD）**：每轮迭代计算梯度时使用全部训练数据；优点是朝着最小值迭代，缺点是样本值很大时每轮更新速度会很慢。
- **随机梯度下降（SGD）**：每轮迭代计算梯度时随机抽取一条训练数据；优点是每轮更新速度快，缺点是样本噪点较多时未必朝着极小值方向更新（多轮迭代能保障整体大致朝着极小值方向更新）。
- **小批量梯度下降（MBGD）**：每轮迭代计算梯度时随机抽取一定比例训练数据；优点是平衡更新梯度方向准确性与更新速度，缺点是batch值较难确定。

虽然随机性可以很好的跳过局部最优值，所以比较推荐使用小批量梯度下降法。由于随机梯度下降较难能达到最小值，可使用逐渐降低学习率缓和这一问题，这个过程被称为模拟退火。

以上虽然只使用了相对简单的假设进行举例，但相同的梯度下降法可以推广到更高维，更复杂的损失函数中使用。

由于函数的计算的复杂性，可导性问题，主要使用梯度下降，参见3.2.2.2（负似然对数与交叉信息熵的梯度形式：$X^T*(h_\theta(x)-Y)$）。

## 3.2 非参数化模型


---
参考资料：
- [Sklearn 与 TensorFlow 机器学习实用指南](https://www.bookstack.cn/books/hands_on_Ml_with_Sklearn_and_TF)
- [最小二乘法](https://zhuanlan.zhihu.com/p/38128785)
- [极大似然估计与最大后验概率估计](https://zhuanlan.zhihu.com/p/40024110)
- [如何通俗的解释交叉熵与相对熵](https://www.zhihu.com/question/41252833)
- [波士顿房价预测任务](https://www.paddlepaddle.org.cn/tutorials/projectdetail/1515038#anchor-13)